In [ ]:
### Implementation:

# Loading
# - Load raw data from csv files
# - Filter rows and columns





# TODO Data

### load csv raw (train, test)
# crop / resample data
# scale data

# load csv filter (train[-idx], train[idx])
# load test / train / valid multiple sources
# split data
# apply transformation on data
# apply aggregation on data
# augment data
# fit model
# predict model
# evaluate model
# save model
# load model
# reporting
# logging
# aggregate models (ensemble, stacking, mean)
# documentation
# Include automatic NA management
# TODO >>>>>> Enhance and refactor csvLoader for headers and multiple headers detection
# Add more format for transformer instanciation
# generate a dataset id dependant of data to avoid hash different with same data but different path - replace hashcode after loading ?

In [6]:
import os


def browse_folder_and_generate_txt(root_folder, extension, output_file):
    with open(output_file, 'w', encoding='utf-8') as out_file:
        for foldername, subfolders, filenames in os.walk(root_folder):
            for filename in filenames:
                if filename.endswith(extension):
                    file_path = os.path.join(foldername, filename)
                    try:
                        with open(file_path, 'r', encoding='utf-8') as file:
                            content = file.read()
                        # Write file path and content to output file
                        out_file.write(f"File: {file_path}\n")
                        out_file.write("\n" + "-"*10 + "\n")
                        out_file.write(content + "\n")
                        out_file.write("\n" + "="*80 + "\n\n")  # Separator
                    except Exception as e:
                        print(f"Error reading {file_path}: {e}")


root_folder = """D:/Workspace/ML/NIRS/pinard/pinard/core"""
extension = "py"
output_file = "core_code.txt"

browse_folder_and_generate_txt(root_folder, extension, output_file)

In [7]:
def generate_files_from_txt(txt_file_path):
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    current_file_path = None
    current_code_chunk = []

    for line in lines:
        if line.startswith('# '):
            if current_file_path and current_code_chunk:
                # if current_file_path is a path with folder, create folders if they don't exist
                if '/' in current_file_path:
                    os.makedirs(os.path.dirname(current_file_path), exist_ok=True)
                
                # Write the current chunk to the current file
                with open(current_file_path, 'a', encoding='utf-8') as code_file:
                    code_file.write(''.join(current_code_chunk))
                current_code_chunk = []
            
            # Update the current file path
            current_file_path = line[2:].strip()
        else:
            current_code_chunk.append(line)
    
    # Write the last chunk to the last file
    if current_file_path and current_code_chunk:
        with open(current_file_path, 'a', encoding='utf-8') as code_file:
            code_file.write(''.join(current_code_chunk))

# Example usage
txt_file_path = "test_code.txt"  # Use the existing variable
generate_files_from_txt(txt_file_path)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

# Define custom transformers for sample and feature augmentation


class AddOneTransformer(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X + 1


class DivTwoTransformer(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X / 2


class AddThreeTransformer(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X + 3


# Sample data
X = np.array([[1, 2, 3], [10, 20, 30], [100, 200, 300]])

from sklearn.preprocessing import MinMaxScaler
# Define the pipeline
pipeline = [
    MinMaxScaler(),
    # {'sample_augmentation': [None, AddOneTransformer(), DivTwoTransformer()]},
    # {'feature_augmentation': [None, AddThreeTransformer()]},
]

# Create the executor and execute the pipeline
executor = PipelineExecutor(pipeline)
executor.execute(X)

# Retrieve data with default aggregation
data = executor.get_data()
print("Concatenated Data:")
print(data)

# Retrieve data with 'union' aggregation
data_union = executor.get_data(aggregation='union')
print("\nUnion Aggregated Data:")
print(data_union)

In [ ]:
import csv
import gzip
import zipfile
from typing import List
from collections import Counter

def read_csv_first_lines(file_path: str) -> List[str]:
    lines = []
    
    if file_path.endswith('.gz'):
        with gzip.open(file_path, mode='rt', newline='', encoding='utf-8') as f:
            reader = f.readlines()
            for i, row in enumerate(reader):
                lines.append(row.strip())
                if i >= 4:  # We only want the first 5 lines
                    break
    elif file_path.endswith('.zip'):
        with zipfile.ZipFile(file_path, 'r') as zip_file:
            # Assuming there is only one CSV file in the zip
            csv_filename = zip_file.namelist()[0]
            with zip_file.open(csv_filename) as f:
                reader = f.read().decode('utf-8').splitlines()
                for i, row in enumerate(reader):
                    lines.append(row.strip())
                    if i >= 4:
                        break
    elif file_path.endswith('.csv'):
        with open(file_path, mode='r', newline='', encoding='utf-8') as f:
            reader = f.readlines()
            for i, row in enumerate(reader):
                lines.append(row.strip())
                if i >= 4:
                    break
    else:
        raise ValueError("Unsupported file format")
    
    return lines

def detect_delimiter(lines: List[str]) -> str:
    delimiters = [',', ';', '\t', '|']
    delimiter_counts = {d: [] for d in delimiters}

    for line in lines:
        for delimiter in delimiters:
            delimiter_counts[delimiter].append(line.count(delimiter))

    most_likely_delimiter = None
    for delimiter, counts in delimiter_counts.items():
        if len(set(counts)) == 1:  # Same count across all lines
            if most_likely_delimiter is None or sum(counts) > sum(delimiter_counts[most_likely_delimiter]):
                most_likely_delimiter = delimiter

    if most_likely_delimiter is None:
        raise ValueError("Could not detect a consistent delimiter")
    
    return most_likely_delimiter

# Example usage
if __name__ == "__main__":
    csv_path = "example.csv"  # Change to your file path
    first_lines = read_csv_first_lines(csv_path)
    delimiter = detect_delimiter(first_lines)
    print(f"Detected delimiter: {delimiter}")


In [ ]:
import csv
import gzip
import zipfile
from typing import List, Union
from collections import Counter
import re
import numpy as np
import pandas as pd

def read_csv_first_lines(file_path: str, n_lines: int) -> List[str]:
    lines = []

    if file_path.endswith('.gz'):
        with gzip.open(file_path, mode='rt', newline='', encoding='utf-8') as f:
            reader = f.readlines()
            for i, row in enumerate(reader):
                lines.append(row.strip())
                if i >= n_lines:  # We only want the first 5 lines
                    break
    elif file_path.endswith('.zip'):
        with zipfile.ZipFile(file_path, 'r') as zip_file:
            # Assuming there is only one CSV file in the zip
            csv_filename = zip_file.namelist()[0]
            with zip_file.open(csv_filename) as f:
                reader = f.read().decode('utf-8').splitlines()
                for i, row in enumerate(reader):
                    lines.append(row.strip())
                    if i >= n_lines:
                        break
    elif file_path.endswith('.csv'):
        with open(file_path, mode='r', newline='', encoding='utf-8') as f:
            reader = f.readlines()
            for i, row in enumerate(reader):
                lines.append(row.strip())
                if i >= n_lines:
                    break
    else:
        raise ValueError("Unsupported file format")

    return lines


def detect_delimiter(lines: List[str], numeric_delimiter: str) -> str:
    delimiters = [';', '\t', '|', ',']
    if numeric_delimiter in delimiters:
        delimiters.remove(numeric_delimiter)

    delimiter_counts = {d: [] for d in delimiters}

    for line in lines:
        for delimiter in delimiters:
            delimiter_counts[delimiter].append(line.count(delimiter))

    most_likely_delimiter = None
    for delimiter, counts in delimiter_counts.items():
        if len(set(counts)) == 1:  # Same count across all lines
            # print(delimiter, sum(counts), sum(delimiter_counts[most_likely_delimiter]) if most_likely_delimiter is not None else None)
            if sum(counts) > 0:
                if most_likely_delimiter is None or sum(counts) > sum(delimiter_counts[most_likely_delimiter]):
                    most_likely_delimiter = delimiter

    return most_likely_delimiter


def detect_numeric_delimiter(lines):
    numeric_delimiters = ['.', ',']
    numeric_delimiter_counts = {numeric_delimiter: 0 for numeric_delimiter in numeric_delimiters}

    for line in lines:
        for numeric_delimiter in numeric_delimiters:
            numeric_delimiter_counts[numeric_delimiter] += len(re.findall(re.escape(numeric_delimiter), line))

    detected_numeric_delimiter = max(numeric_delimiter_counts, key=numeric_delimiter_counts.get)
    
    if numeric_delimiter_counts[detected_numeric_delimiter] == 0:
        detected_numeric_delimiter = None

    return detected_numeric_delimiter


def is_float(value: str) -> bool:
    try:
        float(value)
        return True
    except ValueError:
        return False
    
def is_int(value: str) -> bool:
    try:
        int(value)
        return True
    except ValueError:
        return False
    
def is_bool(value: str) -> bool:
    return value.lower() in ['true', 'false']


def count_fragments_by_type(line: Union[str, List[str]], separator: str) -> dict:
    fragment_types = {'float': 0, 'int': 0, 'str': 0, 'bool': 0}
    fragment_types_list = get_fragment_types(line, separator)
    for fragment_type in fragment_types_list:
        fragment_types[fragment_type] += 1
    
    return fragment_types


def get_fragment_types(line: Union[str, List[str]], separator: str) -> List[str]:
    fragments = []
    if isinstance(line, list):
        fragments = line
    else:
        fragments = line.split(separator)
    
    fragment_types = []
    for fragment in fragments:
        fragment = fragment.strip()
        if is_int(fragment):
            fragment_types.append('int')
        elif is_float(fragment):
            fragment_types.append('float')
        elif is_bool(fragment):
            fragment_types.append('bool')
        else:
            fragment_types.append('str')
    return fragment_types


def detect_column_header(lines: List[str], delimiter: str) -> bool:
    columns = list(zip(*[line.split(delimiter) for line in lines]))
    last_types = None
    current_types = None
    for i, column in reversed(list(enumerate(columns))):
        column = list(column)
        if len(column) < 4:
            raise ValueError("Error: Too few lines to detect header")
        column = column[3:] # remove potential horizontal header
        types = get_fragment_types(column, delimiter)
        
        
    #     if types.count('float') + types.count('int') == len(types):
    #         return False
    # return True
    
    print(np.array(columns).shape)

# def detect_horizontal_header(lines: List[str], delimiter: str) -> bool:
#     types_of_lines = {tuple(sorted(count_fragments_by_type(line, delimiter).items())) for line in lines}
    
#     if len(types_of_lines) > 2:
#         print(types_of_lines)
#         return "Error: Too many lines to detect header"
#     elif len(types_of_lines) == 1:
#         return False
#     else:
#         return True




                
def process_folders_and_detect_delimiters(folder_paths: List[str]):
    for folder_path in folder_paths:
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.endswith('.csv') or file.endswith('.gz') or file.endswith('.zip'):
                    file_path = os.path.join(root, file)
                    print(file_path)
                    try:
                        first_lines = read_csv_first_lines(file_path, n_lines=15)
                        numeric_delimiter = detect_numeric_delimiter(first_lines)
                        delimiter = detect_delimiter(first_lines, numeric_delimiter)
                        # first_line_types = count_fragments_by_type(first_lines[0], delimiter)
                        # second_line_types = count_fragments_by_type(first_lines[1], delimiter)
                        # horizontal_header = detect_horizontal_header(first_lines, delimiter)
                        detect_column_header(first_lines, delimiter)
                        # print(f"File: {file_path}, Numeric: {numeric_delimiter}, Delimiter: {delimiter}, Horizontal Header: {horizontal_header}")
                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")
                return




# Example usage
folder_paths = ['FOOD_CHEM_lost']
process_folders_and_detect_delimiters(folder_paths)


# # Example usage
# if __name__ == "__main__":
#     csv_path = "example1.csv"  # Change to your file path
#     first_lines = read_csv_first_lines(csv_path)
#     delimiter = detect_delimiter(first_lines)
#     print(f"Detected delimiter: {delimiter}")

In [53]:
import os

def check_subfolders_for_epub(root_folder):
    for foldername, subfolders, filenames in os.walk(root_folder):
        has_epub = any(filename.endswith('.epub') for filename in filenames)
        if not has_epub:
            print(f"No .epub files in: {foldername}")

# Example usage
check_subfolders_for_epub("Z:/light_novels/")

No .epub files in: Z:/light_novels/


In [1]:
import os
import shutil

def copy_and_reorganize_files(source_path, new_source_path):
    for root, dirs, files in os.walk(source_path):
        for file in files:
            # Get the relative path from the source_path
            relative_path = os.path.relpath(root, source_path)
            # Split the relative path into parts
            parts = relative_path.split(os.sep)
            if len(parts) >= 2:
                author = parts[0]
                serie = parts[1]
                # Create the new directory structure
                new_dir = os.path.join(new_source_path, f"{author}_{serie}")
                os.makedirs(new_dir, exist_ok=True)
                # Copy the file to the new directory
                shutil.copy(os.path.join(root, file), os.path.join(new_dir, file))


# Example usage
source_path = "D:/calibre_sorted"
new_source_path = "D:/calibre_cpy"
copy_and_reorganize_files(source_path, new_source_path)